Implement dropout algorithms in your neural network.


In [2]:
class SimpleNN:
    def __init__(self, input_dim, hidden_dim, output_dim, drop_prob=0.5, lr=0.01):
        self.W1 = np.random.randn(input_dim, hidden_dim) * 0.01
        self.b1 = np.zeros((1, hidden_dim))
        self.W2 = np.random.randn(hidden_dim, output_dim) * 0.01
        self.b2 = np.zeros((1, output_dim))
        self.dropout = Dropout(drop_prob)
        self.lr = lr

    def relu(self, z):
        return np.maximum(0, z)

    def relu_deriv(self, z):
        return (z > 0).astype(float)

    def forward(self, X, training=True):
        # Layer 1
        self.z1 = X.dot(self.W1) + self.b1
        self.a1 = self.relu(self.z1)

        # Apply dropout here during training
        self.a1_drop = self.dropout.forward(self.a1, training)

        # Layer 2 (output)
        self.z2 = self.a1_drop.dot(self.W2) + self.b2
        return self.z2  # linear output (for regression)

    def backward(self, X, y, out):
        m = y.shape[0]
        dZ2 = (out - y) / m
        dW2 = self.a1_drop.T.dot(dZ2)
        db2 = np.sum(dZ2, axis=0, keepdims=True)

        dA1 = dZ2.dot(self.W2.T)
        dA1 = self.dropout.backward(dA1)  # pass through dropout
        dZ1 = dA1 * self.relu_deriv(self.z1)

        dW1 = X.T.dot(dZ1)
        db1 = np.sum(dZ1, axis=0, keepdims=True)

        # Parameter updates
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2

    def train(self, X, y, epochs=100):
        for epoch in range(epochs):
            out = self.forward(X, training=True)
            loss = np.mean((out - y) ** 2)
            self.backward(X, y, out)
            if epoch % 10 == 0:
                print(f"Epoch {epoch:03d} | Loss: {loss:.4f}")



In [3]:
# Generate simple data: y = 2x + 1 + noise
np.random.seed(42)
X = np.random.randn(200, 1)
y = 2 * X + 1 + np.random.randn(200, 1) * 0.1

model = SimpleNN(input_dim=1, hidden_dim=32, output_dim=1, drop_prob=0.3, lr=0.05)
model.train(X, y, epochs=100)


Epoch 000 | Loss: 4.3542
Epoch 010 | Loss: 3.7941
Epoch 020 | Loss: 3.5774
Epoch 030 | Loss: 3.4489
Epoch 040 | Loss: 3.2510
Epoch 050 | Loss: 2.8682
Epoch 060 | Loss: 2.1972
Epoch 070 | Loss: 1.2750
Epoch 080 | Loss: 0.7325
Epoch 090 | Loss: 0.3912
